In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Étape 1 : Charger le dataset
file_path = r"dataset.csv"  # Mettez ici le chemin correct de votre fichier CSV
try:
    dataset = pd.read_csv(file_path)
    print("Dataset chargé avec succès !")
except FileNotFoundError:
    raise FileNotFoundError(f"Le fichier spécifié n'existe pas à l'emplacement : {file_path}")

# Étape 2 : Afficher les colonnes du dataset pour identifier la colonne cible
print("\nColonnes disponibles dans le dataset :")
print(dataset.columns)

# Demander à l'utilisateur de saisir le nom exact de la colonne cible
target_column = input("\nEntrez le nom exact de la colonne cible (par exemple 'popularity') : ")
# Vérifier si la colonne cible existe
if target_column not in dataset.columns:
    raise ValueError(f"La colonne cible '{target_column}' n'existe pas dans le dataset. Veuillez vérifier.")


# Étape 3 : Aperçu des données
print("\nAperçu des premières lignes du dataset :")
print(dataset.head())
print("\nInformations sur le dataset :")
print(dataset.info())

# Étape 4 : Vérification des valeurs manquantes
missing_values = dataset.isnull().sum()
print("\nValeurs manquantes par colonne :")
print(missing_values)

# Étape 5 : Traitement des colonnes non numériques et valeurs manquantes
# Identifier les colonnes non numériques
non_numeric_columns = dataset.select_dtypes(include=['object']).columns
if len(non_numeric_columns) > 0:
    print("\nColonnes non numériques détectées :", list(non_numeric_columns))
    # Remplir les colonnes non numériques avec une valeur par défaut (exemple : "Inconnu")
    dataset[non_numeric_columns] = dataset[non_numeric_columns].fillna("Inconnu")


# Remplir les colonnes numériques avec leur moyenne
numeric_columns = dataset.select_dtypes(include=['float64', 'int64']).columns
if len(numeric_columns) > 0:
    dataset[numeric_columns] = dataset[numeric_columns].fillna(dataset[numeric_columns].mean())

print("\nValeurs manquantes traitées.")

# **Étape 6 : Statistiques Descriptives**
print("\nStatistiques descriptives :")
print(dataset[numeric_columns].describe())

# Supprimer la colonne 'Unnamed: 0' si elle est présente
dataset = dataset.drop(columns=['Unnamed: 0'], errors='ignore')


# Séparer les features (X) et la target (y)
X = dataset.drop(columns=[target_column])
y = dataset[target_column]



# Sélectionner uniquement les colonnes numériques pour X
X_numeric = X.select_dtypes(include=['float64', 'int64'])



# Étape 7 : Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)  # Normaliser uniquement les colonnes numériques


# Étape 8 : Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Étape 9 : Résumé des ensembles
print("\nTaille de l'ensemble d'entraînement :", X_train.shape)
print("Taille de l'ensemble de test :", X_test.shape)
print("\nPréparation des données terminée avec succès !")


# 1. Analyse de la corrélation (en excluant les colonnes non numériques)
numeric_data = dataset.select_dtypes(include=['float64', 'int64'])  # Ne conserver que les colonnes numériques
correlation_matrix = numeric_data.corr()


# Afficher la matrice de corrélation avec un heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Matrice de Corrélation")
plt.show()

# Afficher la corrélation de chaque feature avec la target
correlation_with_target = correlation_matrix[target_column].sort_values(ascending=False)
print("\nCorrélation avec la variable cible :")
print(correlation_with_target)

# 2. Sélection de K meilleures features avec SelectKBest
# Appliquer l'imputation et sélectionner les K meilleures caractéristiques
imputer = SimpleImputer(strategy='mean')  # Imputer les valeurs manquantes par la moyenne
X_imputed = imputer.fit_transform(X_numeric)  # Imputer les colonnes numériques


# Sélectionner les K meilleures features (par exemple, les 5 meilleures)
selector = SelectKBest(score_func=f_regression, k=5)  # Sélectionner les 5 meilleures features
X_new = selector.fit_transform(X_imputed, y)


# Vous pouvez ensuite accéder aux indices des meilleures caractéristiques :
selected_columns = X_numeric.columns[selector.get_support()]
print("Les meilleures caractéristiques sélectionnées : ", selected_columns)


#Créer des features supplémentaires
# Calcul de la popularité moyenne par genre
genre_popularity = dataset.groupby('track_genre')['popularity'].mean()

# Ajout de la popularité pondérée par genre à la dataset
dataset['popularity_weighted_by_genre'] = dataset['track_genre'].map(genre_popularity)


# Calcul du rapport énergie/danceabilité
dataset['energy_to_danceability_ratio'] = dataset['energy'] / dataset['danceability']


# Conversion de la durée de millisecondes à minutes
dataset['duration_normalized'] = dataset['duration_ms'] / 60000  # 1 minute = 60000 millisecondes


# Initialiser le modèle de régression linéaire
lr_model = LinearRegression()

# Entraîner le modèle avec l'ensemble d'entraînement
lr_model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = lr_model.predict(X_test)

# Calculer les métriques de performance
initial_mse = mean_squared_error(y_test, y_pred)
initial_r2 = r2_score(y_test, y_pred)

print("\n=== Résultats du Modèle Initial ===")
print(f"Erreur Quadratique Moyenne (MSE) : {initial_mse:.2f}")
print(f"Score R² : {initial_r2:.2f}")


# Exemple d'utilisation de GridSearchCV pour vérifier si normaliser les données améliore les performances
param_grid = {
    'fit_intercept': [True, False],  # Inclure ou non l'intercept
    # 'normalize': [True, False],  # Normalisation des données - This parameter is removed
}

grid_search = GridSearchCV(
    estimator=LinearRegression(),
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# Meilleurs hyperparamètres
best_params = grid_search.best_params_
print("\n=== Meilleurs Hyperparamètres ===")
print(best_params)

# Réentraîner le modèle avec les meilleurs paramètres
lr_model_optimized = LinearRegression(**best_params)

# Entraîner le modèle optimisé
lr_model_optimized.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_optimized = lr_model_optimized.predict(X_test)

# Calculer les métriques
optimized_mse = mean_squared_error(y_test, y_pred_optimized)
optimized_r2 = r2_score(y_test, y_pred_optimized)

print("\n=== Résultats du Modèle Optimisé ===")
print(f"Erreur Quadratique Moyenne (MSE) : {optimized_mse:.2f}")
print(f"Score R² : {optimized_r2:.2f}")


# Visualisation des prédictions vs valeurs réelles
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_optimized, alpha=0.7, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.title("Prédictions vs Valeurs Réelles (Modèle Optimisé)")
plt.xlabel("Valeurs Réelles")
plt.ylabel("Prédictions")
plt.show()


# Distribution des résidus
residuals = y_test - y_pred_optimized
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, color='purple', alpha=0.7)
plt.title("Distribution des Résidus")
plt.xlabel("Résidu")
plt.ylabel("Fréquence")
plt.show()


# Évaluation croisée
from sklearn.model_selection import cross_val_score # Import the cross_val_score function

cv_scores = cross_val_score(lr_model_optimized, X_train, y_train, cv=5, scoring='r2')
print("Scores R² par pli de validation croisée : ", cv_scores)
print("Score R² moyen : ", cv_scores.mean())